In [8]:
import pandas as pd
import numpy as np
import os

path = '../data/processed/2010_2020/'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(file))
            
df = pd.DataFrame()

for i,file in enumerate(files):
    print(i,file)
    df_close = pd.read_csv(path + file,index_col=0)['Adj Close']
    df_close.name = file.split('.')[0]
    df = pd.concat([df,df_close],axis=1,join='outer')
    
print(df.shape)

0 riv.csv
1 csco.csv
2 pri.csv
3 gef-b.csv
4 slim.csv
5 bios.csv
6 nzf.csv
7 hlne.csv
8 unt.csv
9 hubs.csv
10 gpl.csv
11 csbk.csv
12 sbgi.csv
13 ufcs.csv
14 gfed.csv
15 airt.csv
16 nxn.csv


//anaconda3/envs/insight/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



17 bny.csv
18 rhe.csv
19 issc.csv
20 kndi.csv
21 eft.csv
22 ustb.csv
23 trns.csv
24 xoxo.csv
25 xshd.csv
26 bci.csv
27 caro.csv
28 arii.csv
29 ntz.csv
30 aeis.csv
31 ual.csv
32 trow.csv
33 isrg.csv
34 egp.csv
35 efc.csv
36 eex.csv
37 nvr.csv
38 ddls.csv
39 plym.csv
40 feim.csv
41 hgsd.csv
42 wor.csv
43 prgo.csv
44 leco.csv
45 rpt.csv
46 vcyt.csv
47 usac.csv
48 hylb.csv
49 oiil.csv
50 cohn.csv
51 nbw.csv
52 plow.csv
53 prgx.csv
54 erf.csv
55 ifn.csv
56 usat.csv
57 fihd.csv
58 phg.csv
59 uve.csv
60 jhmh.csv
61 bud.csv
62 orrf.csv
63 idhd.csv
64 susa.csv
65 pfl.csv
66 gem.csv
67 gdi.csv
68 pds.csv
69 pgh.csv
70 oesx.csv
71 splg.csv
72 bldp.csv
73 nog.csv
74 bxc.csv
75 rmgn.csv
76 gfa.csv
77 ksm.csv
78 chco.csv
79 lbtyk.csv
80 spok.csv
81 splp.csv
82 tpr.csv
83 tsi.csv
84 pbr-a.csv
85 ahl.csv
86 css.csv
87 dvn.csv
88 mgee.csv
89 amsc.csv
90 svt.csv
91 stk.csv
92 xlrn.csv
93 swp.csv
94 dtq.csv
95 scyx.csv
96 mn.csv
97 fxjp.csv
98 asys.csv
99 afam.csv
100 inse.csv
101 dtf.csv
102 esgf.csv
10

681 ndra.csv
682 srlp.csv
683 nvg.csv
684 lnc.csv
685 eyeg.csv
686 rxdx.csv
687 vtnr.csv
688 edi.csv
689 bkyi.csv
690 uan.csv
691 hyxe.csv
692 casi.csv
693 bbrg.csv
694 ntx.csv
695 capr.csv
696 nyh.csv
697 bld.csv
698 fnwb.csv
699 ric.csv
700 pqg.csv
701 allt.csv
702 frgi.csv
703 grf.csv
704 gsb.csv
705 psx.csv
706 vmot.csv
707 ppc.csv
708 wve.csv
709 spyd.csv
710 wtm.csv
711 ctek.csv
712 boh.csv
713 prk.csv
714 ako-a.csv
715 ppt.csv
716 pso.csv
717 rhp.csv
718 else.csv
719 kfy.csv
720 gpn.csv
721 rfap.csv
722 hubb.csv
723 mcep.csv
724 ddbi.csv
725 srcl.csv
726 ppg.csv
727 abax.csv
728 rig.csv
729 imte.csv
730 wwe.csv
731 site.csv
732 gold.csv
733 nvcr.csv
734 mdca.csv
735 tech.csv
736 pro.csv
737 ewgs.csv
738 hefv.csv
739 aubn.csv
740 linc.csv
741 blw.csv
742 wwr.csv
743 wti.csv
744 wyy.csv
745 bndc.csv
746 irm.csv
747 tenx.csv
748 arkg.csv
749 bcx.csv
750 nuo.csv
751 flca.csv
752 rdus.csv
753 bap.csv
754 wrlsu.csv
755 casm.csv
756 bbk.csv
757 flbr.csv
758 mdly.csv
759 efr.csv
760 res

1313 cez.csv
1314 aqua.csv
1315 cfa.csv
1316 fast.csv
1317 xpl.csv
1318 fmdg.csv
1319 mlab.csv
1320 ori.csv
1321 dac.csv
1322 grvy.csv
1323 grub.csv
1324 omex.csv
1325 ffwm.csv
1326 xbio.csv
1327 sor.csv
1328 fsbc.csv
1329 ul.csv
1330 ttai.csv
1331 veco.csv
1332 hope.csv
1333 whlr.csv
1334 bxmt.csv
1335 fosl.csv
1336 cubi.csv
1337 jw-b.csv
1338 bomn.csv
1339 smmv.csv
1340 snv.csv
1341 smm.csv
1342 acer.csv
1343 medp.csv
1344 tjx.csv
1345 mnta.csv
1346 wd.csv
1347 asb.csv
1348 thg.csv
1349 fcel.csv
1350 sna.csv
1351 meet.csv
1352 jcs.csv
1353 hxl.csv
1354 aeo.csv
1355 ai.csv
1356 mpc.csv
1357 toca.csv
1358 br.csv
1359 aft.csv
1360 shen.csv
1361 fbm.csv
1362 d.csv
1363 afc.csv
1364 sanw.csv
1365 gene.csv
1366 hli.csv
1367 atra.csv
1368 ca.csv
1369 mrk.csv
1370 s.csv
1371 xin.csv
1372 esgu.csv
1373 cwco.csv
1374 tri.csv
1375 viv.csv
1376 club.csv
1377 cqh.csv
1378 crs.csv
1379 rnva.csv
1380 hbb.csv
1381 amrc.csv
1382 grbk.csv
1383 aks.csv
1384 cpl.csv
1385 ahh.csv
1386 ln.csv
1387 tsm.csv

...

3126 crnt.csv
3127 tap.csv
3128 fmbh.csv
3129 tbk.csv
3130 caj.csv
3131 efoi.csv
3132 ccu.csv
3133 lvin.csv
3134 tco.csv
3135 htbx.csv
3136 ava.csv
3137 mar.csv
3138 bmra.csv
3139 px.csv
3140 mbi.csv
3141 sc.csv
3142 xyl.csv
3143 waas.csv
3144 landp.csv
3145 atax.csv
3146 amnb.csv
3147 dht.csv
3148 cmi.csv
3149 rp.csv
3150 tnh.csv
3151 hmny.csv
3152 tol.csv
3153 chscp.csv
3154 clm.csv
3155 awr.csv
3156 st.csv
3157 ati.csv
3158 fro.csv
3159 kmpr.csv
3160 faln.csv
3161 fb.csv
3162 aamc.csv
3163 vbtx.csv
3164 gf.csv
3165 sqns.csv
3166 mtd.csv
3167 lmfa.csv
3168 var.csv
3169 aobc.csv
3170 sogo.csv
3171 gprk.csv
3172 numv.csv
3173 veri.csv
3174 cndf.csv
3175 wsky.csv
3176 cxh.csv
3177 dezu.csv
3178 bf-b.csv
3179 glop.csv
3180 expo.csv
3181 ssd.csv
3182 hei.csv
3183 hgv.csv
3184 ttf.csv
3185 esba.csv
3186 alx.csv
3187 ke.csv
3188 glog.csv
3189 gntx.csv
3190 cuo.csv
3191 ftxd.csv
3192 ocx.csv
3193 vuzi.csv
3194 sph.csv
3195 miln.csv
3196 dlph.csv
3197 kr.csv
3198 myc.csv
3199 alo.csv
3200 hi.

3731 acp.csv
3732 rell.csv
3733 dm.csv
3734 inxn.csv
3735 ofix.csv
3736 ttph.csv
3737 hall.csv
3738 fdeu.csv
3739 vntr.csv
3740 igem.csv
3741 bokf.csv
3742 cuda.csv
3743 pacb.csv
3744 virt.csv
3745 vtc.csv
3746 fcal.csv
3747 sd.csv
3748 cof.csv
3749 amov.csv
3750 tsco.csv
3751 shi.csv
3752 crbp.csv
3753 crak.csv
3754 virc.csv
3755 mlhr.csv
3756 asgn.csv
3757 mab.csv
3758 ph.csv
3759 efbi.csv
3760 tsbk.csv
3761 sim.csv
3762 evgn.csv
3763 acbi.csv
3764 dysl.csv
3765 civb.csv
3766 meoh.csv
3767 hska.csv
3768 lsvx.csv
3769 bswn.csv
3770 lbrdk.csv
3771 sgf.csv
3772 cba.csv
3773 mlr.csv
3774 mbin.csv
3775 mmv.csv
3776 cce.csv
3777 cldt.csv
3778 nghc.csv
3779 hst.csv
3780 atlo.csv
3781 taro.csv
3782 ayi.csv
3783 pcti.csv
3784 gnca.csv
3785 hyacu.csv
3786 clgx.csv
3787 jbht.csv
3788 hsc.csv
3789 hsii.csv
3790 axl.csv
3791 ccs.csv
3792 tci.csv
3793 iesc.csv
3794 mpvd.csv
3795 evlmc.csv
3796 bstc.csv
3797 pzza.csv
3798 cal.csv
3799 educ.csv
3800 ccd.csv
3801 rndm.csv
3802 schk.csv
3803 tsla.csv


4333 rgt.csv
4334 egn.csv
4335 edu.csv
4336 bah.csv
4337 buse.csv
4338 ndsn.csv
4339 mchx.csv
4340 isf.csv
4341 prpo.csv
4342 aumn.csv
4343 jasn.csv
4344 eeq.csv
4345 rfp.csv
4346 egy.csv
4347 rgc.csv
4348 hbnc.csv
4349 utsi.csv
4350 bbd.csv
4351 fult.csv
4352 nuw.csv
4353 lngr.csv
4354 nssc.csv
4355 ctic.csv
4356 yum.csv
4357 bpmp.csv
4358 prsc.csv
4359 jhma.csv
4360 ncz.csv
4361 wms.csv
4362 cdzi.csv
4363 litb.csv
4364 eqt.csv
4365 gseu.csv
4366 igc.csv
4367 tile.csv
4368 alty.csv
4369 icad.csv
4370 bti.csv
4371 buz.csv
4372 ghc.csv
4373 igt.csv
4374 eqc.csv
4375 lyv.csv
4376 tannz.csv
4377 utes.csv
4378 fnlc.csv
4379 lbtyb.csv
4380 wbk.csv
4381 plab.csv
4382 agio.csv
4383 eros.csv
4384 ophc.csv
4385 llex.csv
4386 sbra.csv
4387 prim.csv
4388 pdm.csv
4389 trup.csv
4390 pei.csv
4391 ges.csv
4392 elec.csv
4393 prkr.csv
4394 lub.csv
4395 flxs.csv
4396 rsls.csv
4397 mvis.csv
4398 rvsb.csv
4399 ebix.csv
4400 kss.csv
4401 ntnx.csv
4402 uslm.csv
4403 pfe.csv
4404 cswc.csv
4405 kpti.csv
4406 

4938 wmgi.csv
4939 xlre.csv
4940 foe.csv
4941 hbm.csv
4942 ajx.csv
4943 reed.csv
4944 aic.csv
4945 jpgb.csv
4946 csx.csv
4947 gudb.csv
4948 mgen.csv
4949 sach.csv
4950 hci.csv
4951 eccy.csv
4952 dgicb.csv
4953 for.csv
4954 ffbc.csv
4955 trq.csv
4956 crk.csv
4957 edgw.csv
4958 ait.csv
4959 cqp.csv
4960 ni.csv
4961 bmlp.csv
4962 ofc.csv
4963 cpt.csv
4964 layn.csv
4965 vkq.csv
4966 gnrx.csv
4967 tsu.csv
4968 wcfb.csv
4969 fnv.csv
4970 sngx.csv
4971 nvta.csv
4972 pgc.csv
4973 mtrn.csv
4974 lts.csv
4975 kemq.csv
4976 nms.csv
4977 spns.csv
4978 pfg.csv
4979 true.csv
4980 gef.csv
4981 vamo.csv
4982 nclh.csv
4983 hesm.csv
4984 ggn.csv
4985 rdnt.csv
4986 ctso.csv
4987 blfs.csv
4988 gsit.csv
4989 kldw.csv
4990 cstm.csv
4991 nauh.csv
4992 rvrs.csv
4993 udbi.csv
4994 jhmt.csv
4995 opgn.csv
4996 lxp.csv
4997 utf.csv
4998 nap.csv
4999 cohr.csv
5000 bwg.csv
5001 cdxc.csv
5002 epr.csv
5003 zion.csv
5004 pkw.csv
5005 uuu.csv
5006 buff.csv
5007 agen.csv
5008 jhmc.csv
5009 bwp.csv
5010 dzsi.csv
5011 pegi

5542 rcky.csv
5543 tndm.csv
5544 nok.csv
5545 splk.csv
5546 wab.csv
5547 spmo.csv
5548 aieq.csv
5549 nmt.csv
5550 supv.csv
5551 ctsh.csv
5552 ltc.csv
5553 coda.csv
5554 trvn.csv
5555 usod.csv
5556 hvt-a.csv
5557 ksa.csv
5558 fly.csv
5559 amrk.csv
5560 qlc.csv
5561 hmst.csv
5562 ofs.csv
5563 tse.csv
5564 cwbc.csv
5565 jynt.csv
5566 dwaq.csv
5567 azpn.csv
5568 fnf.csv
5569 swk.csv
5570 dtj.csv
5571 crl.csv
5572 wsbf.csv
5573 mu.csv
5574 trv.csv
5575 jpem.csv
5576 insi.csv
5577 cps.csv
5578 lq.csv
5579 nuem.csv
5580 mksi.csv
5581 dwlv.csv
5582 cgix.csv
5583 hni.csv
5584 scvl.csv
5585 msp.csv
5586 aa.csv
5587 aeg.csv
5588 brk-b.csv
5589 l.csv
5590 lcnb.csv
5591 cncr.csv
5592 agx.csv
5593 adc.csv
5594 embu.csv
5595 ipwr.csv
5596 eman.csv
5597 oxfd.csv
5598 ovid.csv
5599 emcf.csv
5600 sfhy.csv
5601 aep.csv
5602 msg.csv
5603 mrc.csv
5604 cnce.csv
5605 ago.csv
5606 ci.csv
5607 jpin.csv
5608 pdli.csv
5609 acrs.csv
5610 oxlco.csv
5611 atri.csv
5612 fbr.csv
5613 aqb.csv
5614 cif.csv
5615 mgu.csv


In [10]:
lag = 1
df_return = (np.log10(df.shift(-lag))-np.log10(df))[::lag]

# if nan (e.g. stop trading, not listed etc), return will be 0
df_return = df_return.fillna(0)

print(df_return)

df_autocorr = df_return.apply(lambda col: col.autocorr(1),axis=0)
df_autocorr = df_autocorr.sort_values(ascending=False)

print(df_autocorr)

//anaconda3/envs/insight/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log10
  
//anaconda3/envs/insight/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log10
  


                 riv      csco       pri     gef-b      slim      bios  \
Date                                                                     
2010-01-04  0.000000 -0.001939  0.000000  0.003360  0.000000 -0.006508   
2010-01-05  0.000000 -0.002836  0.000000  0.012536  0.000000 -0.024379   
2010-01-06  0.000000  0.001952  0.000000 -0.004248  0.000000  0.004247   
2010-01-07  0.000000  0.002296  0.000000 -0.004042  0.000000  0.003158   
2010-01-08  0.000000 -0.001235  0.000000  0.003631  0.000000  0.014950   
...              ...       ...       ...       ...       ...       ...   
2019-12-24  0.001607  0.000636 -0.002001 -0.005311  0.002203  0.001195   
2019-12-26  0.001334 -0.000727 -0.004761  0.003548  0.000957  0.007100   
2019-12-27  0.000000 -0.001640 -0.000266  0.002098 -0.004408 -0.003536   
2019-12-30  0.000533  0.003364  0.000033 -0.000838  0.002436  0.007043   
2019-12-31  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

                 nzf      hlne       